In [ ]:
from Bio.PDB import PDBParser, MMCIFParser, Superimposer
from Bio.PDB.PDBExceptions import PDBConstructionException
import os

def load_structure(file_path):
    """
    Loads a structure from a given file path, automatically selecting the
    correct Biopython parser (PDB or MMCIF) based on the file extension.
    """
    # Determine the file format and select the appropriate parser
    if file_path.lower().endswith(('.pdb', '.ent')):
        parser = PDBParser(QUIET=True)
    elif file_path.lower().endswith(('.cif', '.mmcif')):
        parser = MMCIFParser(QUIET=True)
    else:
        raise ValueError("Unsupported file format. Please use .pdb, .ent, .cif, or .mmcif.")

    # Use the filename (without extension) as the structure ID
    structure_id = os.path.splitext(os.path.basename(file_path))[0]

    try:
        # get_structure returns a Biopython Structure object
        structure = parser.get_structure(structure_id, file_path)
        print(f"Successfully loaded structure '{structure_id}' from {file_path}")
        return structure
    except (IOError, PDBConstructionException) as e:
        print(f"Error loading {file_path}: {e}")
        return None

def get_ordered_heavy_atoms(structure):
    """
    Extracts a list of all heavy (non-hydrogen) atoms from the structure's
    first model, maintaining the sequential order based on residue and atom ID.
    This ordered list is crucial for a valid RMSD comparison.

    Heavy atoms are filtered based on the element not being 'H' (Hydrogen).
    """
    heavy_atoms = []

    # We only calculate RMSD for the first model (index 0)
    try:
        model = structure[0]
    except IndexError:
        print("Error: Structure contains no models.")
        return []

    for chain in model:
        for residue in chain:
            # Skip common non-core residues like water (HOH/WAT) and solvent/ions
            if residue.get_resname() in ('HOH', 'WAT'):
                continue

            for atom in residue.get_atoms():
                # Check for the element explicitly being non-Hydrogen.
                # Biopython usually populates the 'element' field.
                element = atom.element

                if element is None or element == '':
                    # Fallback check if 'element' field is not populated:
                    # check if the atom name starts with 'H' (PDB convention)
                    if atom.get_name()[0] != 'H':
                        heavy_atoms.append(atom)
                elif element != 'H':
                    heavy_atoms.append(atom)

    return heavy_atoms

def calculate_rmsd_heavy_atoms(pdb_file_path, mmcif_file_path):
    """
    Loads two structures, extracts their ordered heavy atoms, checks for
    consistency, superimposes them, and calculates the RMSD.
    """
    # 1. Load the structures
    structure_ref = load_structure(pdb_file_path)
    structure_mov = load_structure(mmcif_file_path)

    if structure_ref is None or structure_mov is None:
        return None

    # 2. Extract heavy atoms
    atoms_ref = get_ordered_heavy_atoms(structure_ref)
    atoms_mov = get_ordered_heavy_atoms(structure_mov)

    # 3. Validation
    if not atoms_ref or not atoms_mov:
        print("RMSD Error: Could not find heavy atoms in one or both structures.")
        return None

    if len(atoms_ref) != len(atoms_mov):
        print("\n--- RMSD CANCELED ---")
        print(f"Error: Mismatch in the number of heavy atoms to compare.")
        print(f"Atoms in PDB file: {len(atoms_ref)}")
        print(f"Atoms in mmCIF file: {len(atoms_mov)}")
        print("The two structures must contain the exact same number of heavy atoms in the same order for a valid comparison.")
        return None

    print(f"\nComparing {len(atoms_ref)} heavy atoms...")

    # 4. Superimpose and calculate RMSD
    super_imposer = Superimposer()
    # Align atoms_mov (mobile) onto atoms_ref (reference)
    super_imposer.set_atoms(atoms_ref, atoms_mov)
    super_imposer.run()

    # Get the resulting RMSD value
    rmsd = super_imposer.get_rmsd()

    # 5. Output results
    print("\n--- RMSD Results ---")
    print(f"RMSD (Heavy Atoms): {rmsd:.3f} Angstroms")

    return rmsd

if __name__ == "__main__":
    # --- USER CONFIGURATION ---
    # !!! REPLACE these placeholders with the actual paths to your files !!!
    PDB_FILE = "path/to/your/structure.pdb"
    MMCIF_FILE = "path/to/your/structure.cif"
    # Example for actual files:
    # PDB_FILE = "1a2b.pdb"
    # MMCIF_FILE = "1a2b.cif"

    # Check if placeholder paths are still used and provide a warning
    if PDB_FILE.startswith("path/to") or MMCIF_FILE.startswith("path/to"):
        print("!!! IMPORTANT: Please update PDB_FILE and MMCIF_FILE variables with your actual file paths. !!!")
        print("Current script will exit as placeholder paths cannot be loaded.")
    else:
        # Run the RMSD calculation
        calculate_rmsd_heavy_atoms(PDB_FILE, MMCIF_FILE)